In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

weights = tf.Variable(tf.random.normal([300, 200], stddev=0.5),
                          name="weights")

weights_untrainable = tf.Variable(tf.random.normal([300, 200], stddev=0.5),
                          name="weights", trainable=False)

Instructions for updating:
non-resource variables are not supported in the long term


`tf.random.normal ` produces a tensor initialized using a normal distribution with standard deviation 0.5

---
the size of the tensor i.e. `300, 200` indicates that the weights connect a layer with 300 neurons to a layer with 200 neurons

---
the `trainable` option results in gradients being automatically computed and applied to `weights`


In [2]:
# Other common methods for creating tensors

shape = [300, 200]
sample_weights_zeros = tf.Variable(tf.zeros(shape, dtype=tf.float32, name=None))
sample_weights_ones = tf.Variable(tf.ones(shape, dtype=tf.float32, name=None))
samples_trunc_norm = tf.Variable(tf.random.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None))
sample_rnd_uni = tf.Variable(tf.random.uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None))


when `tf.Variable` is called, three operations are added to the computation graph: the operation producing the tensor (e.g. `tf.zeros`), the `tf.assign` operation which initializes the tensor, and the variable operation (e.g. `weights`) which stores the current value

In [3]:
# x represents a minibatch of data stored as float32s
x = tf.placeholder(tf.float32, name="x", shape=[None, 784])   # None indicates that x can be initialized with any number of samples, 
                                                              # 784 indicates that each data sample has 784 dimensions
W = tf.Variable(tf.random.uniform([784, 10], -1, -1), name="W") 
multiply = tf.matmul(x, W)

`tf.placeholder` enables creating the computation graph without data

example below of reusing network and layer functions

In [4]:
tf.disable_v2_behavior()
def layer(input, weight_shape, bias_shape):
  weight_init = tf.random_uniform_initializer(minval=-1, maxval=1)
  bias_init = tf.constant_initializer(value=0)
  W = tf.get_variable("W", weight_shape, initializer=weight_init)
  b = tf.get_variable("b", bias_shape, initializer=bias_init)
  return tf.matmul(input, W) + b

def sample_network(input):
  with tf.variable_scope("layer_1"):
    output_1 = layer(input, [784, 100], [100])
  with tf.variable_scope("layer_2"):
    output_2 = layer(output_1, [100, 50], [50])
  with tf.variable_scope("layer_3"):
    output_3 = layer(output_2, [50, 10], [10])
  return output_3

i_1 = tf.placeholder(tf.float32, [1000, 784], name="i_1")
sample_network(i_1)

<tf.Tensor 'layer_3/add:0' shape=(1000, 10) dtype=float32>

In [ ]:
def inference(x):
  tf.constant_initializer(value=0)
  W = tf.get_variable("W", [784, 10], initializer=init)
  b = tf.get_variable("b", [10], initializer=init)
  output = tf.nn.softmax(tf.matmul(x, W) + b)
  return output

def loss(output, y):
  dot_product = y * tf.log(output)
  x_entropy = -tf.reduce_sum(dot_product, reduction_indices=1)
  loss = tf.reduce_mean(x_entropy)
  return loss

def training(cost, global_step):
  tf.scalar_summary("cost", cost)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  train_op = optimizer.minimize(cost, global_step=global_step)
  return train_op

def evaluate(output, y):
  correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  return accuracy

learning_rate = 0.01
training_epochs = 1000
batch_size = 100
display_step = 1

with tf.Graph().as_default():
  x = tf.placeholder("float", [None, 784]) # Input has dimension of 784 i.e. images of shape 28*28
  y = tf.placeholder("float", [None, 10])  # Output consists of 10 classes
  output = inference(x)
  cost = loss(output, y)
  global_step = tf.Variable(0, name='global_step', trainable=False)
  train_op = training(cost, global_step)
  eval_op = evaluate(output, y)
  summary_op = tf.merge_all_summaries()
  saver = tf.train.Saver()
  sess = tf.Session()
  summary_writer = tf.train.SummaryWriter("logistic_logs/", graph_def=sess.graph_def)
  init_op = tf.initialize_all_variableS()
  sess.run(init_op)
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size) # i.e. number of batches
    for i in range(total_batch):
      mbatch_x, mbatch_y = mnist.train.next_batch(batch_size)
      feed_dict = {x: mbatch_x, y: mbatch_y}
      sess.run(train_op, feed_dict=feed_dict)
      minibatch_cost = sess.run(cost, feed_dict=feed_dict)
      avg_cost += minibatch_cost/total_batch
    # log per epoch
    if epoch % display_step == 0:
      val_feed_dict = {
        x : mnist.validation.images,
        y : mnist.validation.labels
      }
      accuracy = sess.run(eval_op, feed_dict=val_feed_dict)
      summary_str = sess.run(summary_op, feed_dict=feed_dict)
      summary_writer.add_summary(summary_str, sess.run(global_step))
      saver.save(sess, "logistic_logs/model-checkpoint", global_step=global_step)
  test_feed_dict = {
      x : mnist.test.images,
      y : mnist.test.labels
  }
  accuracy = sess.run(eval_op, feed_dict=test_feed_dict)

